In [1]:
import requests, urllib3
import threading
import time,os
urllib3.disable_warnings()

In [2]:
http = urllib3.PoolManager(maxsize=100)

In [3]:
chunk = 10240
thread_no = 20

In [4]:
url = '''https://r2---sn-o3o-qxaz.googlevideo.com/videoplayback?key=yt6&source=youtube&mime=video/webm&pl=19&fvip=4&signature=B9216673B1CD0611137B77A8E74D972328A5694E.BB89E734763F9F33A3CAB1654508F316AFFFB084&ei=-Vz6W4mSFJK4qAGJqZ_4Bg&itag=272&gir=yes&sparams=aitags,clen,dur,ei,gir,id,initcwndbps,ip,ipbits,itag,keepalive,lmt,mime,mm,mn,ms,mv,pl,requiressl,source,expire&initcwndbps=1172500&id=o-AJk732y2bqsOLagxLc5DzouHMODPwLIPYz_ZVaoYxQyJ&mm=31,29&ipbits=0&c=WEB&mn=sn-o3o-qxaz,sn-qxa7sn7r&ms=au,rdu&mt=1543134356&dur=659.100&mv=m&clen=1847336750&ip=14.139.228.222&keepalive=yes&lmt=1537659543094242&aitags=133,134,135,136,137,160,242,243,244,247,248,271,272,278,313&requiressl=yes&expire=1543156057&alr=yes&cpn=mcWZfNul35AWufbt&cver=2.20181117&range=0-1847336750'''
r = http.request('HEAD',url)
file_size = int(r.headers['content-length'])
print(file_size)

1847336750


In [5]:
def download_part(start,end):
    global http
    headers = {'Range': 'bytes=%d-%d' % (start, end)}
    r = http.request('GET',url, headers=headers, preload_content=False)
    with open('.'+str(start)+'-'+str(end),'wb') as f:
        for c in r.stream(chunk):
            if c:
                f.write(c)

In [6]:
if file_size<1000000:
    thread_no = 1
dl = int(file_size/thread_no)
st_en = []
for i in range(thread_no):
    st_en.append([dl*i,(dl*i)+(dl-1)])
st_en[-1][1]=file_size

In [7]:
tm = time.time()
th=[]
for i in range(thread_no):
    th.append(threading.Thread(target=download_part,args=(st_en[i][0],st_en[i][1])))
    th[i].start()
for i in th:
    i.join()
print(time.time()-tm)

130.1183145046234


In [8]:
with open('file_name.mp4','wb') as f:
    for i in st_en:
        f.write(open('.'+str(i[0])+'-'+str(i[1]),'rb').read())
        os.remove('.'+str(i[0])+'-'+str(i[1]))